In [2]:
import mymodel
import tensorflow as tf
from glob import glob
import os
import cv2
from PIL import Image
from keras.utils import normalize
import numpy as np



Using TensorFlow backend.


In [4]:
img_dataset = []
msk_dataset = []

f_img = sorted(glob('dataset/train/*'))
for f in f_img:
    image = cv2.imread(f)
    image = Image.fromarray(image)
    # image = image.resize((SIZE, SIZE))
    img_dataset.append(np.array(image))

f_msk = sorted(glob('dataset/validation/*'))
for f in f_msk:
    image = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    (thresh, im_bw) = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    image = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY)[1]
    image = Image.fromarray(image)
    # image = image.resize((SIZE, SIZE))
    msk_dataset.append(np.array(image))

# img_dataset = (img_dataset - v_min)/(v_max - v_min)
# print(img_dataset.shape)
# #Normalize images
img_dataset = normalize(np.array(img_dataset), axis=1)
# #D not normalize masks, just rescale to 0 to 1.
msk_dataset = (np.array(msk_dataset)) /255.

In [5]:
print(img_dataset.shape, msk_dataset.shape)

(731, 200, 200, 3) (731, 200, 200)


In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
X_train, x_test, y_train, y_test = train_test_split(img_dataset, msk_dataset, test_size= 0.20, random_state=0)
# y_train = to_categorical(y_train, 3)

In [7]:
print(X_train.shape, y_train.shape)

(584, 200, 200, 3) (584, 200, 200)


## DATA SANITY CHECK

In [ ]:
# import random
# import numpy as np
# import matplotlib.pyplot as plt
# X_train_quick_test, X_test_quick_test, y_train_quick_test, y_test_quick_test = train_test_split(X_train, y_train, test_size = 0.9, random_state = 0)
# image_number = random.randint(0, len(X_train_quick_test))
# plt.figure(figsize=(12, 6))
# plt.subplot(121)
# plt.imshow(np.reshape(X_train_quick_test[image_number], (200, 200,3)), cmap='gray')
# plt.subplot(122)
# plt.imshow(np.reshape(y_train_quick_test[image_number], (200, 200,3)), cmap='gray')
# plt.show()

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout,Conv2DTranspose, concatenate
from tensorflow.keras.optimizers import Adam
    
# model = Sequential()
# model.add(Conv2D(filters =32, kernel_size = (32,32), padding = "same", activation = "relu", input_shape = INP))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (3,3)))
# model.add(Conv2D(filters = 32, kernel_size = (32,32), padding = "same", activation = "relu", input_shape = (100,100)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(Conv2D(filters = 32, kernel_size = (32,32), padding = "same", activation = "relu", input_shape = (50, 50)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(Conv2D(filters = 32, kernel_size = (32,32), padding = "same", activation = "relu", input_shape = (25, 25)))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(Flatten())
# model.add(Dropout(0.7))
# model.add(Dense(64, activation = "relu"))
# model.add(BatchNormalization())
# # model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(400, activation = "sigmoid"))

# adam = Adam(lr = 0.001)
# model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ['accuracy','mse'])

# model.summary()
nbr_data = img_dataset.shape[0]
epoch = 2000
input_w = 256
input_h = 256
input_ch = 3

s  = Input((input_w, input_h, input_ch))
#Downscale path -----------------------------------------------------------------------------------
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = Dropout(0.1)(c1)
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = MaxPooling2D((2, 2))(c1)
#100
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = Dropout(0.1)(c2)
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = MaxPooling2D((2, 2))(c2)
#50
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = Dropout(0.2)(c3)
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = MaxPooling2D((2, 2))(c3)
#15
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = Dropout(0.2)(c4)
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = Dropout(0.3)(c5)
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Scale up path ----------------------------------------------------------------------------------
u6 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = Dropout(0.2)(c6)
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
    
u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = Dropout(0.2)(c7)
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
    
u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = Dropout(0.1)(c8)
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
    
u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = Dropout(0.1)(c9)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
    
outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
model = Model(inputs=[s], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_72 (Conv2D)              (None, 256, 256, 16) 448         input_7[0][0]                    
__________________________________________________________________________________________________
dropout_34 (Dropout)            (None, 256, 256, 16) 0           conv2d_72[0][0]                  
__________________________________________________________________________________________________
conv2d_73 (Conv2D)              (None, 256, 256, 16) 2320        dropout_34[0][0]                 
____________________________________________________________________________________________

In [16]:
history = model.fit(X_train, 
                    y_train,
                    verbose = 1,
                    batch_size = 32,
                    epochs = epoch,
                    validation_data = (x_test, y_test),
                    shuffle = False)
model.save("Asagao30m")

2022-01-20 17:34:37.377629: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 280320000 exceeds 10% of free system memory.


Epoch 1/2000


2022-01-20 17:34:58.824456: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-20 17:34:58.851074: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3393735000 Hz


ValueError: in user code:

    /home/capilab/miniconda3/envs/cv_asagau/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/capilab/miniconda3/envs/cv_asagau/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/capilab/miniconda3/envs/cv_asagau/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/capilab/miniconda3/envs/cv_asagau/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/capilab/miniconda3/envs/cv_asagau/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(

In [12]:
def cal_result(n, f, s, ch, p = 1):
    w = h = ((n + 2*p - f)/s) + 1
    n = ch
    d = ((n + 2*p - f)/s) + 1
    print("Output = [{} {} {}]".format(w,h,d))

In [14]:
cal_result(200,3,1,3)

Output = [204.0 204.0]
